In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates

In [12]:
df = pd.read_csv("supplier_ranking_grades.csv")

In [13]:
#Scaling to the scale 0-10
df['delivery time'] = df['delivery time'] * 2
df['flexibility'] = df['flexibility'] * 2
df['conditions and method of payment'] = df['conditions and method of payment'] * 2
df['quality'] = df['quality'] * 2


In [15]:
df1 = df.drop(["supplier's location and traffic connections"], axis =1)

Using KNN Clustering to classify suppliers into ideal or not

In [16]:
df2 = df1

X = df2.drop(['supplier'], axis=1)

# Scale the features to have mean=0 and standard deviation=1, except for quantity and price
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[X_scaled.columns.difference(['quantity', 'price'])] = scaler.fit_transform(X[X_scaled.columns.difference(['quantity', 'price'])])

# Convert quantity to a rating on a scale of 0-10 (higher values are better)
X_scaled['quantity'] = (X_scaled['quantity'] - X_scaled['quantity'].min()) / (X_scaled['quantity'].max() - X_scaled['quantity'].min()) * 10

# Perform k-means clustering with 2 clusters (ideal and not ideal)
kmeans = KMeans(n_clusters=2, random_state=42)
df2['cluster'] = kmeans.fit_predict(X_scaled)

In [17]:
print(df2[['supplier', 'cluster']])

                           supplier  cluster
0                   Romaguera Group        1
1                    Mayert-Shields        1
2                 Runolfsdottir LLC        1
3                      Connelly Inc        1
4                      Lowe-Bartell        1
5         Kilback, Swift and Larson        1
6                  Feil-Schamberger        1
7                    Gutmann-Murray        1
8             Oberbrunner-Abernathy        1
9      Pollich, Trantow and Streich        1
10                Hessel-Rutherford        1
11      Ledner, Schneider and Jerde        0
12                 Konopelski-Green        1
13                     Ferry-Heaney        0
14                   Abshire-Deckow        1
15                     Jones-Rippin        0
16       Schmidt, Dicki and Quigley        0
17       Emard, Strosin and Goodwin        1
18                Weissnat and Sons        1
19                   Ondricka-Adams        0
20                       Kuphal LLC        1
21        

Finding Final Scores of a supplier

In [18]:
from sklearn.preprocessing import MinMaxScaler
features = ['quality', 'conditions and method of payment', 'serviceability and communicativeness of the supplier',
            'reputation of the supplier and its competence', 'flexibility', 'financial condition of the supplier',
            'condition of the supplier\'s assets', 'business results and number of employees', 'price', 'delivery time']

# Define equal feature weights for all parameters (sum of weights is 1)
equal_weight = 1 / len(features)
feature_weights = {feature: equal_weight for feature in features}

# Normalize the features using Min-Max scaling
scaler = MinMaxScaler()
df_normalized = scaler.fit_transform(df[features])

# Calculate the score for each supplier
df['score'] = (df_normalized * list(feature_weights.values())).sum(axis=1)

df_sorted = df.sort_values(by='score', ascending=False)
print(df_sorted[['supplier', 'score']])


                           supplier     score
8             Oberbrunner-Abernathy  0.733889
3                      Connelly Inc  0.704444
13                     Ferry-Heaney  0.658889
23                        Nolan Inc  0.646667
2                 Runolfsdottir LLC  0.631111
9      Pollich, Trantow and Streich  0.629444
12                 Konopelski-Green  0.627778
15                     Jones-Rippin  0.607222
10                Hessel-Rutherford  0.578333
25  Schiller, Luettgen and O'Conner  0.577222
24                    Casper-Waters  0.562778
26                 Runolfsson Group  0.533333
5         Kilback, Swift and Larson  0.530000
11      Ledner, Schneider and Jerde  0.510000
27      Zboncak, Hackett and Corwin  0.507222
28                     Graham-Lakin  0.481111
32                     Conroy Group  0.449444
14                   Abshire-Deckow  0.447778
20                       Kuphal LLC  0.447778
18                Weissnat and Sons  0.442222
22                       Becker LL